In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path
from bisect import bisect
from itertools import chain
from collections import Counter
from tqdm import tqdm
from collections import defaultdict

In [ ]:
from unicode import unicode_mapping
from preprocess_v2 import NormalizerSGZChat

In [ ]:
pd.set_option('display.max_rows', 10**6)
pd.set_option('display.max_colwidth', -1)

## Load

In [ ]:
path_data = Path('~/data/yk-sgz2017-chat/').expanduser()

In [ ]:
path_cwd = path_data / 'data-7-20210120'

## EDA

### unicode range

In [ ]:
def str_to_unicode(s):
    return [r'\u{:04X}'.format(ord(c)) for c in s]
    # return [ord(c) for c in s]
    # return [r'{:04X}'.format(ord(c)) for c in s]
    # return ['{:04X}'.format(ord(c)) for c in s]
    # return ['\u{:04X}'.format(ord(c)) for c in s]

In [ ]:
s = df.head(1).content.tolist()[0]
for _ in str_to_unicode(s):
    print(_)

count by range

In [ ]:
_, breakpoints, names = zip(*unicode_mapping)

names = list(names)
names.append('unknown')
names = tuple(names)

In [ ]:
cnt = Counter()

In [ ]:
charset = defaultdict(set)

In [ ]:
def halfwidth_and_fullwidth_forms_to_basic_latin(c):
    code = ord(c)
    if 0xff01 <= code and 0xff5e >= code:
        return chr(code-0xfee0)
    return c

In [ ]:
def index(c) -> int:
    code = ord(c)
    i = bisect(breakpoints, code)
    return i-1 if code == breakpoints[i-1] else i

In [ ]:
def unicode_range(s, breakpoints=breakpoints, names=names):
    # return [bisect(breakpoints, c) for c in s]
    r = []
    for c in s:
        c = halfwidth_and_fullwidth_forms_to_basic_latin(c)
        i = index(c)
        charset[i].add(c)
        r.append(i)
    return r

In [ ]:
for _ in tqdm(path_cwd.iterdir()):
    df = pd.read_csv(_, usecols=['content'])
    cnt.update(Counter(chain.from_iterable(df.content.apply(unicode_range).tolist())))
    # break
    
#cnt

In [ ]:
for k, v in sorted(cnt.items()):
    print(k, v, names[k])

In [ ]:
for k, v in sorted(cnt.items()):
    print('-' * 80)
    print(k, v, names[k])
    print()
    # if k not in [0, 71, 78, 82, 106, 119, 306]:
    print(charset[k])
    print('-' * 80)

In [ ]:
for c in charset[71]:
    print(c, r'\u%04x'%ord(c))

In [ ]:
def has(s, i):
    return i in unicode_range(s)

In [ ]:
def get(s, i):
    return [s[j] for j, _ in enumerate(unicode_range(s)) if i == _]

表情
- Latin-1 Supplement
- Latin Extended-A
- IPA Extensions
- Spacing Modifier Letters
- Combining Diacritical Marks
- Greek and Coptic
- Cyrillic
- Cyrillic Supplement
- Armenian
- Arabic
- Devanagari
- Bengali
- Gurmukhi
- Gujarati
- Oriya
- Telugu
- Kannada
- Sinhala
- Thai
- Lao
- Tibetan
- Georgian
- Hangul Jamo
- Unified Canadian Aboriginal Syllabics
- Balinese
- Phonetic Extensions
- Combining Diacritical Marks Supplement
- Greek Extended
- Superscripts and Subscripts
- Arrows
- Mathematical Operators
- Optical Character Recognition
- Box Drawing
- Block Elements
- Geometric Shapes
- Supplemental Punctuation
- Hiragana
- Katakana （也有用作分隔符的时候）
- Hangul Compatibility Jamo
- Yi Syllables
- Yi Radicals
- Hangul Syllables
- CJK Compatibility Forms
- Small Form Variants
- Arabic Presentation Forms-B
- Halfwidth and Fullwidth Forms（已经完成的全角转半角，剩余部分基本用于表情）
- Miscellaneous Symbols and Pictographs
- Emoticons (Emoji)
- Supplemental Symbols and Pictographs

动作
- Ethiopic
- Tai Le

乘号
- Latin-1 Supplement
- Number Forms

字母
- Spacing Modifier Letters,
    - ˢ -> s
- Cyrillic
    - Ь -> b
- Phonetic Extensions
    - ᵒ -> o
    - ᵏ -> k
    - ᵇ -> b
- Number Forms
    - Ⅴ -> v
    - ⅴ -> v
    - ⅰ -> i
- Mathematical Operators
    - √ -> v
    - ∨ -> v
    
数字
- Cyrillic
    - О -> 0
- Superscripts and Subscripts
    - ⁶ -> 6
- Number Forms
    - ⅱ -> 2
    - Ⅱ -> 2
- Hiragana
    - ち -> 5
   
字母+数字
- Number Forms
    - Ⅵ -> v1
    
汉字
- Arrows
    - ↑ -> 上
    - ↓ -> 下
- Mathematical Operators
    - √ -> 对（可先转v，再根据情境判断）
    - ≡ -> 三（也有不少，用于表情之中）
- Kangxi Radicals
    - ⼃⼂⼁ -> 川 （仅一次）
    
标点符号
- Mathematical Operators
    - ⋯ -> General Punctuation 省略号
- Box Drawing
    - ━ -> - （偶尔起短横线分隔符的作用，也常起表情的作用。可统一转为短横线，不影响表情的分析。）
    
无意义
- CJK Radicals Supplement
    - ⻌
- CJK Strokes
    - ㇏
- Specials
    
时间日期
- CJK Compatibility

其它
- Small Form Variants 转 Basic Latin

In [ ]:
dft = []

for _ in tqdm(path_cwd.iterdir()):
    df = pd.read_csv(_, usecols=['content'])

    # dft.append(df[df.content.apply(lambda s: has(s, 1))])
    df['substring'] = df.content.apply(lambda s: get(s, 162))
    # print(df)
    dft.append(df[df['substring'].apply(lambda x: 0 < len(x))])
    
df = pd.concat(dft, axis=0)
df

## Ref

str -> unicode

[1] https://www.geeksforgeeks.org/python-convert-string-to-unicode-characters/

str format

: 号后面带填充的字符，只能是一个字符，不指定则默认是用空格填充。

b、d、o、x 分别是二进制、十进制、八进制、十六进制。<br>
其中x大小写，区分显示时采用大小写；

^, <, > 分别是居中、左对齐、右对齐，后面带宽度<br>
此处省略，宽度（示例：{:04X}中为4位）

[2] https://www.runoob.com/python/att-string-format.html

In [ ]:
'{:#x}'.format(11)

In [ ]:
'{:#X}'.format(11)

r string, r for raw

表示转义字符时，自动单斜线转为又斜线。便于一些情况下的处理；

[3] https://stackoverflow.com/questions/33729045/what-does-an-r-represent-before-a-string-in-python

In [ ]:
r'\'

In [ ]:
r'\n'

In [ ]:
'\n'

In [ ]:
'\u1234'

全角 -> 半角

TODO: 转换前后没有看出区别

[4] https://www.jianshu.com/p/152e081fec1b